In [2]:
import numpy as np
import pandas as pd
from src.data.load import load_dataset
from src.data.process import extract_targets

data = load_dataset()
for k, v in data.items():
    print(len(v))

1044681
184937
1229618
178297
37960


提取特征

In [3]:
from src.data.features import extract_user_feature, extract_features

user_features = extract_user_feature(data['train'], data['all_uid'])
features_train = extract_features(data['train'], user_features)
features_valid = extract_features(data['valid'], user_features)
features_test = extract_features(data['test'], user_features)
print('feature dim:', features_train.shape[1])
targets_train = extract_targets(data['train'], 'log')
targets_valid = extract_targets(data['valid'], 'linear')

Counting User Histories: 100%|██████████| 1229618/1229618 [00:30<00:00, 40175.64it/s]


feature dim: 34


线性回归 - 拟合+预测

In [4]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

# model = LinearRegression()
model = RandomForestRegressor(n_estimators=50, n_jobs=-1)
model.fit(features_train, targets_train)
predicts = model.predict(features_valid)
predicts.shape

(184937, 3)

In [5]:
from src.metric import compute_metrics

# 计算指标时恢复对数
def exp_interactions(x: np.ndarray) -> np.ndarray:
    """先计算指数，然后取整"""
    return np.rint(np.exp(x) - 1).astype(int)
compute_metrics(exp_interactions(predicts), targets_valid)

{'score': 0.6512342095375061}

测试集结果提交

In [6]:
test_set_predicts = exp_interactions(model.predict(features_test))
results = []
for i in range(len(data['test'])):
    results.append(
        "{}\t{}\t{},{},{}\n".format(
            data['test'].loc[i, 'uid'],
            data['test'].loc[i, 'mid'],
            test_set_predicts[i, 1],
            test_set_predicts[i, 2],
            test_set_predicts[i, 0]
        )
    )
with open("submission.txt", 'w') as f:
    f.writelines(results)